# Youtube Data Harvesting

Manually harvest data from youtube using google youtube API

In [13]:
cd C:\\Users\\Elamparithi\\PycharmProjects\\Utube_DHW_5

C:\Users\Elamparithi\PycharmProjects\Utube_DHW_5


C:\Users\Elamparithi\PycharmProjects\Utube_DHW_5\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


#### Keep Secrets separate

In [14]:
from dotenv import load_dotenv
from os import getenv
load_dotenv('.secrets')

youtube_api_key = getenv('youtube_api_key')

#### import necessary modules

In [16]:
import requests
from re import compile
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [17]:
youtube = build('youtube', 'v3', developerKey=youtube_api_key)

we dont know the channel id to proced to further harvesting. so, following to be commenced.
### Extracting channel ID

In [18]:
channel_name = 'guvi' # Behindwoodstv,    veritasium
request = youtube.search().list(part='snippet', q=channel_name, type='channel', maxResults=1)
response = request.execute()

In [19]:
print(response)

{'kind': 'youtube#searchListResponse', 'etag': 'kygLO6EmhCykSau237ER5UxQyGw', 'nextPageToken': 'CAEQAA', 'regionCode': 'IN', 'pageInfo': {'totalResults': 20659, 'resultsPerPage': 1}, 'items': [{'kind': 'youtube#searchResult', 'etag': 'S1NtNBOdBCdZgDcM9E14AMIGBYw', 'id': {'kind': 'youtube#channel', 'channelId': 'UCduIoIMfD8tT3KoU0-zBRgQ'}, 'snippet': {'publishedAt': '2012-06-20T09:27:56Z', 'channelId': 'UCduIoIMfD8tT3KoU0-zBRgQ', 'title': 'GUVI', 'description': 'GUVI is an IIT-M & IIM-A incubated edu-tech company that offers Wide range of online Tech courses in your Native language.', 'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/jm7j9zyQfWfFW5C-bKCSjs-b5jWRnk2U_1_4GcIwkbf_wGffvR_dVcdn77U5ROuApqrhJN0Zag=s88-c-k-c0xffffffff-no-rj-mo'}, 'medium': {'url': 'https://yt3.ggpht.com/jm7j9zyQfWfFW5C-bKCSjs-b5jWRnk2U_1_4GcIwkbf_wGffvR_dVcdn77U5ROuApqrhJN0Zag=s240-c-k-c0xffffffff-no-rj-mo'}, 'high': {'url': 'https://yt3.ggpht.com/jm7j9zyQfWfFW5C-bKCSjs-b5jWRnk2U_1_4GcIwkbf_wGffvR_dVcdn7

In [20]:
channel_id = response['items'][0]['snippet']['channelId']
print(channel_id)

UCduIoIMfD8tT3KoU0-zBRgQ


## Getting channel Info with channel ID

In [21]:
request = youtube.channels().list(part='snippet,statistics,contentDetails,status, topicDetails', id=channel_id)
response = request.execute()

In [22]:
response_dct = response['items'][0]
print(response_dct) # channel info (ie: title ,description, viewCount, subscriberCount, status) # channel ID is already  available. 

{'kind': 'youtube#channel', 'etag': 'zWs5ZvjYw-nC4CtGOlt3JrFI490', 'id': 'UCduIoIMfD8tT3KoU0-zBRgQ', 'snippet': {'title': 'GUVI', 'description': 'GUVI is an IIT-M & IIM-A incubated edu-tech company that offers Wide range of online Tech courses in your Native language.\nWith 100% placement support, Globally recognized certifications, Mentors from top global product companies  and Platforms improve your Programming and IT skills GUVI does it all. With over 18 Lakh+ learners and 10+ years of experience we provide apex learning through our Edu-tech services, GUVI shares its most loved Tech-videos on YouTube!\n\nWhat’s on our YouTube Channel?\nCareer guidance videos that give an idea for beginners & early professionals to upgrade their careers.\nNew initiatives & offers that unlock your flexible path for learning the latest technologies.\nWalkthrough videos of our practice platforms that shape up your coding & web development skills.\n\n\n\n\n\n\n', 'customUrl': '@guvisharing', 'publishedAt

### Extracting Channel Type
There is not option in youtube API. but there is some thing like topicDetails.topicCategories[]. 
it ia list, For "A list of Wikipedia URLs that describe the channel's content". This is varchar limited by guvi to 255. so, as it being a list remove "https://en.wikipedia.org/wiki/" and only key the topic.

In [23]:
urls = response_dct['topicDetails']['topicCategories']
urls

['https://en.wikipedia.org/wiki/Lifestyle_(sociology)',
 'https://en.wikipedia.org/wiki/Knowledge']

In [24]:
response_dct.keys()

dict_keys(['kind', 'etag', 'id', 'snippet', 'contentDetails', 'statistics', 'topicDetails', 'status'])

In [25]:
def process_urls(url_list):
    prefix = 'https://en.wikipedia.org/wiki/'
    processed_parts = [url.replace(prefix, '') for url in url_list]
    result = ', '.join(processed_parts)
    return result

In [26]:
result = process_urls(urls)
result

'Lifestyle_(sociology), Knowledge'

### Note: Videos that not includeded in any playlist of a channel is available below. 

The channel named "veritasium" has no playlist at all. This is the default youtube playlist created for every youtube channel created by default, no delete. 

In [27]:
uploads_id = response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]
uploads_id

'UUduIoIMfD8tT3KoU0-zBRgQ'

this can be also extracted with same method as other playslist extraction method. but you need to add this in the playlist list.

## Playlsit extraction

In [28]:
request = youtube.playlists().list(part='snippet', channelId=channel_id, maxResults=50)
response = request.execute()# 

In [33]:
response

{'kind': 'youtube#playlistListResponse',
 'etag': 'XFnH5sDSD_hBK5RihPTMRDjLTSs',
 'nextPageToken': 'CDIQAA',
 'pageInfo': {'totalResults': 101, 'resultsPerPage': 50},
 'items': [{'kind': 'youtube#playlist',
   'etag': '2OyWeC4Knx0mbRlGwfzuYwY4B1I',
   'id': 'PL_9uM5be2amraK2Ca9uFLzfTrICM8gNFZ',
   'snippet': {'publishedAt': '2024-10-11T05:36:52.876222Z',
    'channelId': 'UCduIoIMfD8tT3KoU0-zBRgQ',
    'title': 'Learn With GUVI - Season 2',
    'description': '',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/acaRZ8dJqsk/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/acaRZ8dJqsk/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/acaRZ8dJqsk/hqdefault.jpg',
      'width': 480,
      'height': 360},
     'standard': {'url': 'https://i.ytimg.com/vi/acaRZ8dJqsk/sddefault.jpg',
      'width': 640,
      'height': 480},
     'maxres': {'url': 'https://i.ytimg.com/vi/acaRZ

In [34]:
playlist_data = []

request = youtube.playlists().list(part='snippet', channelId=channel_id, maxResults=50)
response = request.execute()
playlist_data.extend(response['items'])
while 'nextPageToken' in response:
    request = youtube.playlists().list(part='snippet', channelId=channel_id, maxResults=50,
                                                 pageToken=response['nextPageToken'])
    response = request.execute()
    playlist_data.extend(response['items'])

In [35]:
playlist_list = []
for c, item in enumerate(playlist_data):
    plid = item['id']
    playlist_list.append(plid)

In [36]:
playlist_list_full=[uploads_id]
playlist_list_full.extend(playlist_list)
playlist_list_full

['UUduIoIMfD8tT3KoU0-zBRgQ',
 'PL_9uM5be2amraK2Ca9uFLzfTrICM8gNFZ',
 'PL_9uM5be2amrNFUn_HyAVYy1BmVW4RqZi',
 'PL_9uM5be2amqfY1RdRvHN-biJINzfSxJ3',
 'PL_9uM5be2ampNTQMxM7lr_j5dcZZFgxLa',
 'PL_9uM5be2amrtYgBeXNDX1D3nhS-CPcO1',
 'PL_9uM5be2amoelBKyrtVy6kDK74ztHJdZ',
 'PL_9uM5be2amoGVT5rqGXFRdQz6Gqu9sWN',
 'PL_9uM5be2amqgdHCG_OStCR8dddQaiR__',
 'PL_9uM5be2amoedmRrYq7A6rkqL95SuPVq',
 'PL_9uM5be2ampk5Vh5WN7mi-VAOjUF8qD5',
 'PL_9uM5be2amq7piq6uTztjw67PGRUi7Ug',
 'PL_9uM5be2amqDOa_4CsWNVhkYEeFbjTFK',
 'PL_9uM5be2amqc6DkYz4k3fS7eOWuRoOAu',
 'PL_9uM5be2amqYCG5KmehgW9gN2xoFZH2Q',
 'PL_9uM5be2ampWjMVHLN2A6GE8cOFGet8G',
 'PL_9uM5be2ampjCzF-3NJxyw9ADWV7otvb',
 'PL_9uM5be2amqgN9r4iIBc9jE-1mDs78fD',
 'PL_9uM5be2amrLc5Xm4bTqbZZrpbYHGpCg',
 'PL_9uM5be2ampn69bi1ieUJgdQ_7xy5oOO',
 'PL_9uM5be2ampSMsjZNzeMw6FHoUgNEJjJ',
 'PL_9uM5be2amoVVWJbsh9vW4Z73y60KgT6',
 'PL_9uM5be2amqv1bjb668MBhV3IedL56r3',
 'PL_9uM5be2amomhAxKBf_6H8kxWEfRGVRC',
 'PL_9uM5be2amomfsl5yxYNXNK4LO84QLMh',
 'PL_9uM5be2amqFLPIHSCmBQJIRjvWqPcq

In [37]:
len(playlist_list_full)

102

In [38]:
for c, item in enumerate(response['items']):
    plid = item['id']
    playlist_list.append(plid)
    print(c, plid, item['snippet']['title'])

0 PL_9uM5be2amr_cmtdwdoxMcRamc7pCQCV hadoop


In [39]:
for item in enumerate(playlist_list):
    print(item)

(0, 'PL_9uM5be2amraK2Ca9uFLzfTrICM8gNFZ')
(1, 'PL_9uM5be2amrNFUn_HyAVYy1BmVW4RqZi')
(2, 'PL_9uM5be2amqfY1RdRvHN-biJINzfSxJ3')
(3, 'PL_9uM5be2ampNTQMxM7lr_j5dcZZFgxLa')
(4, 'PL_9uM5be2amrtYgBeXNDX1D3nhS-CPcO1')
(5, 'PL_9uM5be2amoelBKyrtVy6kDK74ztHJdZ')
(6, 'PL_9uM5be2amoGVT5rqGXFRdQz6Gqu9sWN')
(7, 'PL_9uM5be2amqgdHCG_OStCR8dddQaiR__')
(8, 'PL_9uM5be2amoedmRrYq7A6rkqL95SuPVq')
(9, 'PL_9uM5be2ampk5Vh5WN7mi-VAOjUF8qD5')
(10, 'PL_9uM5be2amq7piq6uTztjw67PGRUi7Ug')
(11, 'PL_9uM5be2amqDOa_4CsWNVhkYEeFbjTFK')
(12, 'PL_9uM5be2amqc6DkYz4k3fS7eOWuRoOAu')
(13, 'PL_9uM5be2amqYCG5KmehgW9gN2xoFZH2Q')
(14, 'PL_9uM5be2ampWjMVHLN2A6GE8cOFGet8G')
(15, 'PL_9uM5be2ampjCzF-3NJxyw9ADWV7otvb')
(16, 'PL_9uM5be2amqgN9r4iIBc9jE-1mDs78fD')
(17, 'PL_9uM5be2amrLc5Xm4bTqbZZrpbYHGpCg')
(18, 'PL_9uM5be2ampn69bi1ieUJgdQ_7xy5oOO')
(19, 'PL_9uM5be2ampSMsjZNzeMw6FHoUgNEJjJ')
(20, 'PL_9uM5be2amoVVWJbsh9vW4Z73y60KgT6')
(21, 'PL_9uM5be2amqv1bjb668MBhV3IedL56r3')
(22, 'PL_9uM5be2amomhAxKBf_6H8kxWEfRGVRC')
(23, 'PL_9uM5be2amomf

# get playlist titles

PL_9uM5be2amrNFUn_HyAVYy1BmVW4RqZi

playlist_title,
playlist_ID,
playlist_description,
playlist_videos
playlist_video_count

In [40]:
request = youtube.playlists().list(part='snippet', 
                                   id='UUduIoIMfD8tT3KoU0-zBRgQ')
response = request.execute()

In [41]:
response

{'kind': 'youtube#playlistListResponse',
 'etag': 'crNI4uVwVA25CYi-LIW4F7z3A40',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#playlist',
   'etag': 'B_URYwY2eeF4mzuPhOwr7eKml_s',
   'id': 'UUduIoIMfD8tT3KoU0-zBRgQ',
   'snippet': {'publishedAt': '2020-02-23T08:00:00Z',
    'channelId': 'UCduIoIMfD8tT3KoU0-zBRgQ',
    'title': 'Uploads from GUVI',
    'description': '',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/5Pvpph0yPX8/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/5Pvpph0yPX8/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/5Pvpph0yPX8/hqdefault.jpg',
      'width': 480,
      'height': 360},
     'standard': {'url': 'https://i.ytimg.com/vi/5Pvpph0yPX8/sddefault.jpg',
      'width': 640,
      'height': 480},
     'maxres': {'url': 'https://i.ytimg.com/vi/5Pvpph0yPX8/maxresdefault.jpg',
      'width': 1280,
      'he

In [42]:
for items in playlist_list:
    request = youtube.playlists().list(part='snippet,contentDetails', id=items)
    response = request.execute()
    playlist_title = response['items'][0]['snippet']['title']
    print(playlist_title)

Learn With GUVI - Season 2
UI - UX Testimonials - ZEN
UI/UX
GEN - AI
GUVI Podcasts
Public Opinion | GUVI
Top 10 Tech News of the week
AI For India 2.0 Webinars
AI for India 2.0
Design Series | GUVI
All about HR
How To | Guvi Series
payITforward | Arun Prakash | PODCAST SERIES | GUVI
Latest Video | Guvi
Shorts | Guvi
Learn with Industry Expert - Webinar
Zen Class by GUVI - Testimonial
Mobile Hacking Tamil
Ethical Hacking in Tamil
Python Programming Course in Tamil
Python programming exercises series #shorts
Test NG #Shorts
Quick Email Tutorials
Flask Series #Shorts
Learn SQL using Squid game series #shorts
Quick Selenium Series #Shorts
Zen
Developerhavelife
#AIforAll
Youtube ads
Pre-Bootcamp-CodeKata-Javascript
HR
Kaggle
programming
Android Development
PR
Technology
Project Manager
System Design
Story Telling
Growth Marketing
Success Stories
Flask-updated
Html Basics 2
Html css
PHP Hindi
Digital Marketing
Data Science
Blockchain
30 Day Webinar
CSS3
Placement Contest Updates
ReactNative


In [43]:
request = youtube.playlists().list(part='snippet,contentDetails', id='UUduIoIMfD8tT3KoU0-zBRgQ')
response = request.execute()
response.keys()

dict_keys(['kind', 'etag', 'pageInfo', 'items'])

In [44]:
response

{'kind': 'youtube#playlistListResponse',
 'etag': '0auMraB715FSfKIRITTfVI2QSl8',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#playlist',
   'etag': 'Ew3ZYOIdszcLELVbl-1xbO0UGk8',
   'id': 'UUduIoIMfD8tT3KoU0-zBRgQ',
   'snippet': {'publishedAt': '2020-02-23T08:00:00Z',
    'channelId': 'UCduIoIMfD8tT3KoU0-zBRgQ',
    'title': 'Uploads from GUVI',
    'description': '',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/5Pvpph0yPX8/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/5Pvpph0yPX8/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/5Pvpph0yPX8/hqdefault.jpg',
      'width': 480,
      'height': 360},
     'standard': {'url': 'https://i.ytimg.com/vi/5Pvpph0yPX8/sddefault.jpg',
      'width': 640,
      'height': 480},
     'maxres': {'url': 'https://i.ytimg.com/vi/5Pvpph0yPX8/maxresdefault.jpg',
      'width': 1280,
      'he

## Get videos from playlist

In [45]:
videos = []
selected_playlist_id = playlist_list[0] # uploads playlist
selected_playlist_id

'PL_9uM5be2amraK2Ca9uFLzfTrICM8gNFZ'

In [46]:
request = youtube.playlistItems().list(part='contentDetails', 
                                       playlistId=selected_playlist_id, 
                                       maxResults=50)

In [47]:
while request:
    response = request.execute()
    req_item = response['items']
    videos.extend(req_item) # using 'extend' adds the entire list of 50(videos)  in one go to videos-list.
    print(req_item)
    break
    request = youtube.playlistItems().list_next(request, response)
    # "list_next" method reloads the request with another 50 videos till exhausted.

[{'kind': 'youtube#playlistItem', 'etag': 'uXfoLcMJ3AbtB3B6XhycVgq3bEY', 'id': 'UExfOXVNNWJlMmFtcmFLMkNhOXVGTHpmVHJJQ004Z05GWi4wOTA3OTZBNzVEMTUzOTMy', 'contentDetails': {'videoId': 'acaRZ8dJqsk', 'videoPublishedAt': '2024-10-15T15:42:25Z'}}]


In [48]:
len(videos)

1

In [49]:
vid_list = [video['contentDetails']['videoId'] for video in videos]
vid_list

['acaRZ8dJqsk']

## Getting video details

In [50]:
def get_video_info(video_id):
    request = youtube.videos().list(
        part='statistics,snippet,contentDetails',
        id=video_id
    )
    response = request.execute()
    return response['items'][0] if response['items'] else None

In [51]:
for counting, ite in enumerate(vid_list):
    tei_output =get_video_info(ite)
    print(counting, tei_output)
    break

0 {'kind': 'youtube#video', 'etag': 'XeIdYFFfer2gg_CvBQYMgA3vwUY', 'id': 'acaRZ8dJqsk', 'snippet': {'publishedAt': '2024-10-15T15:42:25Z', 'channelId': 'UCduIoIMfD8tT3KoU0-zBRgQ', 'title': 'Learn With GUVI is back with Season 2! - 15th October | GUVI', 'description': "In just 7 days, learn to build your very own BookMyShow-style app with hands-on guidance from GUVI's expert mentors.\n\nDon't miss the live session with Raja Vasantham (RV) happening from 7 PM to 9 PM on Monday, 14th October. Get live mentorship, boost your skills, and earn a global certificate to showcase in your portfolio.\n\nRegister for FREE today and start your app development journey! -https://www.guvi.in/mlp/learn-with-guvi-season2\n\n#LearnwithGUVI\n\nSubscribe to our channel - https://youtube.com/@Guvisharing?sub_confirmation=1\n\nWelcome to our Guvi Youtube channel! We are dedicated to providing high-quality educational content that helps learners of all ages and backgrounds achieve their learning goals.\n\nSo, 

In [53]:
vlt_info = tei_output

In [54]:
vlt_info['statistics']['commentCount']

'0'

### check for communt count

In [55]:
Vid_comment_count = vlt_info['statistics']['commentCount']
print(Vid_comment_count, type(Vid_comment_count))

0 <class 'str'>


### filter the result

In [56]:
video_dict = {                          # guvi formatting
    f"Video_Id_{counting}": {
        "Video_Id": vlt_info['id'],
        "Video_Name": vlt_info['snippet']['title'],
        "Video_Description": vlt_info['snippet']['description'],
        "Tags": vlt_info['snippet'].get('tags', []),
        "PublishedAt": vlt_info['snippet']['publishedAt'],
        "View_Count": vlt_info['statistics'].get('viewCount', 0),
        "Like_Count": vlt_info['statistics'].get('likeCount', 0),
        "Dislike_Count": vlt_info['statistics'].get('dislikeCount', 0),
        "Favorite_Count": vlt_info['statistics'].get('favoriteCount', 0),
        "Comment_Count": vlt_info['statistics'].get('commentCount', 0),
        "Duration": vlt_info['contentDetails']['duration'],
        "Thumbnail": vlt_info['snippet']['thumbnails']['default']['url'],
        "Caption_Status": 'Available' if vlt_info['contentDetails']['caption'] == 'true' else "Unavailable",
        #"Comments": comment_data_dict
    }
}

In [57]:
video_dict

{'Video_Id_0': {'Video_Id': 'acaRZ8dJqsk',
  'Video_Name': 'Learn With GUVI is back with Season 2! - 15th October | GUVI',
  'Video_Description': "In just 7 days, learn to build your very own BookMyShow-style app with hands-on guidance from GUVI's expert mentors.\n\nDon't miss the live session with Raja Vasantham (RV) happening from 7 PM to 9 PM on Monday, 14th October. Get live mentorship, boost your skills, and earn a global certificate to showcase in your portfolio.\n\nRegister for FREE today and start your app development journey! -https://www.guvi.in/mlp/learn-with-guvi-season2\n\n#LearnwithGUVI\n\nSubscribe to our channel - https://youtube.com/@Guvisharing?sub_confirmation=1\n\nWelcome to our Guvi Youtube channel! We are dedicated to providing high-quality educational content that helps learners of all ages and backgrounds achieve their learning goals.\n\nSo, if you're ready to take your learning to the next level, hit that subscribe button and join our community of lifelong lear

## Getting comment detials.

In [58]:
selected_video_id = vlt_info['id']
selected_video_id

'acaRZ8dJqsk'

In [59]:
request = youtube.commentThreads().list(
            part='snippet',
            videoId=selected_video_id,
            textFormat='plainText',
            maxResults=10
        )

In [60]:
if Vid_comment_count == '0':
    # For testing use GUVI, It has blocked comments in its videos.
    print(f'There is no comments in this video, comment count {Vid_comment_count}')
else:
    response = request.execute()
    comment_items = response['items'][0]
    print(comment_items)

There is no comments in this video, comment count 0


In [63]:
if not Vid_comment_count:
    comment_data = {
        "Comment_Id": comment_items['id'],
        "Comment_Text": comment_items['snippet']['topLevelComment']['snippet']['textOriginal'],
        "Comment_Author": comment_items['snippet']['topLevelComment']['snippet']['authorDisplayName'],
        "Comment_PublishedAt": comment_items['snippet']['topLevelComment']['snippet']['publishedAt']
    }
    print(comment_data)